In [13]:
import pandas as pd
import numpy as np
import torch
import json
from normalizer import counter
from torch.utils.data import DataLoader


In [14]:
# All of our fields have names, but we need them in numbers, so define a dictionary to convert.
global magDict
magDict = {
    'TOTUSJH': 0,
    'TOTBSQ': 1,
    'TOTPOT': 2,
    'TOTUSJZ': 3,
    'ABSNJZH': 4,
    'SAVNCPP': 5,
    'USFLUX': 6,
    'TOTFZ': 7,
    'MEANPOT': 8,
    'EPSZ': 9,
    'SHRGT45': 10,
    'MEANSHR': 11,
    'MEANGAM': 12,
    'MEANGBT': 13,
    'MEANGBZ': 14,
    'MEANGBH': 15,
    'MEANJZH': 16,
    'TOTFY': 17,
    'MEANJZD': 18,
    'MEANALP': 19,
    'TOTFX': 20,
    'EPSY': 21,
    'EPSX': 22,
    'R_VALUE': 23,
    'RBZ_VALUE': 24,
    'RBT_VALUE': 25,
    'RBP_VALUE': 26,
    'FDIM': 27,
    'BZ_FDIM': 28,
    'BT_FDIM': 29,
    'BP_FDIM': 30,
    'PIL_LEN': 31,
    'XR_MAX': 32
}

In [15]:
# Get the data from the JSON file, then return it as a tensor of input data and a list of labels
def getDataFromJSON(path="data/train_partition1_data.json", device='cpu', earlyStop=-1):
    # I might need to refactor these arguments to allow to get the test data. Problem for another day.
    # path is the path to the files, device is where to store it (CUDA), earlyStop is how many lines to 
    # read if you don't want the entire file read.
    
    # Get the dictionary to assign names to numbers
    global magDict
    
    # This dataset is heavily skewed, so we need to get the number of each type of flare.
    # This also lets us get the number of lines in the file with a sum.
    weights = counter(path, earlyStop)
    lines = np.sum(weights)
    # Check when we want to stop - the end of the file or earlier.
    if earlyStop < 0: length = lines
    else: length = min(earlyStop, lines)
    
    # Get the file and open it. 
    file = open(path)    
    
    # Declare a tensor to hold the data, and a list to hold the labels.
    # Dimensions: 0: number of entries we want. 1: the 33 fields in the data. 2: the 60 observations in each field. 
    tnsr = torch.Tensor().new_empty((length, 33, 60), device=device)
    labels = []
    flares = {'X':0, 'M':1, 'C':2, 'B':3, 'Q':4}
        
    row = -1
    for line in file:
        # Load the line as a dictionary. Row is an integer place and v is a smaller dictionary.
        d: dict = json.loads(line)
        row += 1
        for _, v in d.items(): # we use the _ because we don't want the ID.
            if earlyStop > 0 and row >= earlyStop:
                # If we don't want the entire dataset, stop loading more than we want
                return tnsr, labels
            if row % 100 == 0:
                print(f'Now loading event {row}/{length}')
            # append the label to our list
            labels.append(flares[v['label']])
            
            # Break each individual dictionary into dictionaries of observations
            # Key is the string in magDict, and timeDict is a dictionary of observations over time
            for key, timeDict in v['values'].items():
                # Turn our name string into a numeric value
                location = magDict[key]
                # Get the measurements out of the time series dictionary
                for timeStamp, measurement in timeDict.items():
                    tnsr[row][location][int(timeStamp)] = measurement
    print(f'{row} lines loaded.')
    # Close the file. I'm not a heathen                    
    file.close()
    # This might be a good place to perform some post processing, but that's a question for another day.
    # Famous last words.
    return tnsr, labels, weights



In [16]:
# This file has 77270 data points. 
%time train1, labels1, weights1 = getDataFromJSON(path="data/train_partition1_data.json", earlyStop=77270)

Now loading event 0/77270
Now loading event 100/77270
Now loading event 200/77270
Now loading event 300/77270
Now loading event 400/77270
Now loading event 500/77270
Now loading event 600/77270
Now loading event 700/77270
Now loading event 800/77270
Now loading event 900/77270
Now loading event 1000/77270
Now loading event 1100/77270
Now loading event 1200/77270
Now loading event 1300/77270
Now loading event 1400/77270
Now loading event 1500/77270
Now loading event 1600/77270
Now loading event 1700/77270
Now loading event 1800/77270
Now loading event 1900/77270
Now loading event 2000/77270
Now loading event 2100/77270
Now loading event 2200/77270
Now loading event 2300/77270
Now loading event 2400/77270
Now loading event 2500/77270
Now loading event 2600/77270
Now loading event 2700/77270
Now loading event 2800/77270
Now loading event 2900/77270
Now loading event 3000/77270
Now loading event 3100/77270
Now loading event 3200/77270
Now loading event 3300/77270
Now loading event 3400/772

Now loading event 27700/77270
Now loading event 27800/77270
Now loading event 27900/77270
Now loading event 28000/77270
Now loading event 28100/77270
Now loading event 28200/77270
Now loading event 28300/77270
Now loading event 28400/77270
Now loading event 28500/77270
Now loading event 28600/77270
Now loading event 28700/77270
Now loading event 28800/77270
Now loading event 28900/77270
Now loading event 29000/77270
Now loading event 29100/77270
Now loading event 29200/77270
Now loading event 29300/77270
Now loading event 29400/77270
Now loading event 29500/77270
Now loading event 29600/77270
Now loading event 29700/77270
Now loading event 29800/77270
Now loading event 29900/77270
Now loading event 30000/77270
Now loading event 30100/77270
Now loading event 30200/77270
Now loading event 30300/77270
Now loading event 30400/77270
Now loading event 30500/77270
Now loading event 30600/77270
Now loading event 30700/77270
Now loading event 30800/77270
Now loading event 30900/77270
Now loadin

Now loading event 55100/77270
Now loading event 55200/77270
Now loading event 55300/77270
Now loading event 55400/77270
Now loading event 55500/77270
Now loading event 55600/77270
Now loading event 55700/77270
Now loading event 55800/77270
Now loading event 55900/77270
Now loading event 56000/77270
Now loading event 56100/77270
Now loading event 56200/77270
Now loading event 56300/77270
Now loading event 56400/77270
Now loading event 56500/77270
Now loading event 56600/77270
Now loading event 56700/77270
Now loading event 56800/77270
Now loading event 56900/77270
Now loading event 57000/77270
Now loading event 57100/77270
Now loading event 57200/77270
Now loading event 57300/77270
Now loading event 57400/77270
Now loading event 57500/77270
Now loading event 57600/77270
Now loading event 57700/77270
Now loading event 57800/77270
Now loading event 57900/77270
Now loading event 58000/77270
Now loading event 58100/77270
Now loading event 58200/77270
Now loading event 58300/77270
Now loadin

In [ ]:
# This file has 93767 data points. 
%time train2, labels2, weights2 = getDataFromJSON(path="data/train_partition2_data.json", earlyStop=93767)

In [ ]:
# This file has 42986 data points. 
%time train3, labels3, weights3 = getDataFromJSON(path="data/train_partition3_data.json", earlyStop=42986)

In [ ]:
train2[0,0]

In [ ]:
# Define the network. Make sure to end with nn.Softmax activation
import torch.nn as nn
!pip install skorch
from skorch import NeuralNet

class MyModule(nn.Module):
  def __init__(self, input_size, n, hidden_size, num_classes=5, drop1=.5):
    super().__init__() 
    self.n = n
    self.layer1 = []
    for _ in range(33):
      self.layer1.append(nn.Linear(60,n))
    self.layer2 = nn.Linear(n, hidden_size)
    layerout = nn.Linear(hidden_size, num_classes)
    #Define a RELU Activation unit
    self.relu = nn.ReLU()  
    self.smax = nn.Softmax(dim=1)
    self.drop = nn.Dropout(p=drop1)
    
  def forward(self, x):
    #Forward Propagate through the layers as defined above
    output = torch.tensor(x.shape[0], 33, self.n)
    for i in range(33):
      output[:,i,:]=layer1[i](x[:, i, :])
    y = self.drop(self.relu( output.reshape(-1, 33*self.n)))
    y = self.smax(self.layer2(y))
    y = self.layerout(y)
    return y


In [ ]:
def train(model, inputs, labels, weight, valSets, valLabels, valweight):
    # TODO: Is this right? How do I determine the weights here?
    weight = torch.Tensor(weight)
    lfc = nn.CrossEntropyLoss(weight=1/weight)
    #ideas
    # 1-(weight/np.sum(weight))
    # .2/weight - this one normalizes so that each class is responsible for 20% of the loss
    # 1/weight - this is a bit naive, but the classes with fewer items are weighted more.
    
    # Hyperparameters
    batch = 256
    epochs = 10
    lr = 0.01
    
    # Start a dataloader object
    data = list(zip(inputs,labels))
    val = list(zip(valSets,valLabels))
    loader = DataLoader(data, batch_size = batch, num_workers=4)
    valLoader = DataLoader(val, batch_size = int(n/4), num_workers=4)
    opt = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range epoch:
        model.train()
        batch_loss = []
        for (xtrain,ytrain) in loader:
            output = model(xtrain)
            loss = lfc(output,ytrain)
            opt.zero_grad()
            loss.backward()
            opt.step()
            batchLoss.append(loss.item())
        print(f'The training loss for epoch {epoch+1}/{epochs} was {np.mean(batchLoss)}')
        model.eval()
        for (xval,yval) in valloader:
            output = model(xval)
            loss = lfc(output,yval)
            batchLoss.append(loss.item())
            unbalancedAccuracy = torch.mean(torch.Tensor(yval) == torch.argmax(output))
            
            balanced = torch.Tensor.new_empty((len(yval), 5))
            # TODO: figure this one out.
            for i in range(5)
                if i is not 
            
        print(f'The validation loss was :   {epoch+1}/{epochs} was {np.mean(batchLoss)}')
